In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
import numpy as np
from graphdatascience import GraphDataScience
from py2neo import Graph
from neo4j import GraphDatabase
import networkx as nx
import matplotlib.pyplot as plt
import torch.nn.functional as F
import pandas as pd
import neo4jupyter
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling, remove_self_loops, add_self_loops, to_dense_adj, dense_to_sparse
from torch_geometric.nn import VGAE,GAE
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, average_precision_score
from torch_geometric.utils import to_dense_adj
#neo4jupyter.init_notebook_mode()
import tqdm as notebook_tqdm

In [3]:
import pickle
# Load the list of arrays from the .pkl file
with open('med_node_embeddings_50d.pkl', 'rb') as pkl_file:
    extracted_data = pickle.load(pkl_file)

In [4]:
node_embeddings = extracted_data

In [ ]:
diseases=pd.read_csv('C:\\Users\\krjoob\\Desktop\\Knowledge graph\\UKBB\\UKB_CMR_med\\processing aftre extracting the data\\All_diseases\\nodes and edges\\Disease.csv')
Med=pd.read_csv('C:\\Users\\krjoob\\Desktop\\Knowledge graph\\UKBB\\UKB_CMR_med\\drug_disease.csv')

In [ ]:
diseases_nodes=list(diseases.fname)
medication_nodes=list(medication.fname)

In [ ]:
diseases_embeddings = {key:node_embeddings[key] for key in diseases_nodes if key in node_embeddings}
medication_embeddings = {key:node_embeddings[key] for key in medication_nodes if key in node_embeddings}

In [145]:
HF_ind = ["Dabigatran", "Sotalol", "Warfarin", "Heparin", "Atenolol", "Diltiazem", "Timolol", "Bisoprolol", "Carvedilol", "Propafenone", "Edoxaban", 
 "Betrixaban", "Rivaroxaban", "Apixaban", "Labetalol", "Dofetilide", "Metoprolol", "Propranolol", "Amiodarone", "Clopidogrel", "Quinidine", 
 "Flecainide", "Dronedarone", "Digoxin", "Verapamil", "Procainamide", "Nadolol","Dipyridamole","Esmolol","Acetyldigitoxin"]


In [146]:
HF_cont = ["Bupivacaine", "Methylphenidate", "Cholecalciferol", "Lacosamide", "Alendronic acid", "Dobutamine",
 "Ergocalciferol", "Lidocaine", "Fidaxomicin", "Azithromycin", "Fusidic acid", "Amoxicillin", 
 "Naproxen", "Ibuprofen", "Oxytetracycline", "Tetracycline", "Disopyramide", "Quinine", "Epinephrine", 
 "Conivaptan","Cetirizine","Dorzolamide","Pantoprazole" ,"Loratadine","Raltitrexed","Methadone",
           "Erythromycin", "Clarithromycin"
 ]


In [147]:
exc_sid = ["Protamine", "Iodixanol", "Clopenthixol", "Tigecycline", "Gadobenic acid", "Vidarabine", "Voriconazole", 
           "Ropivacaine", "Bupivacaine", "Mepivacaine", "Levobupivacaine", "Dexmedetomidine", "Dolasetron", "Granisetron", 
           "Indapamide", "Milrinone", "Cinacalcet", "Hyoscyamine", "Chloroprocaine", "Eplerenone", "Ibutilide", "Alfuzosin", 
           "Phenylephrine", "Droperidol", "Thioridazine", "Zuclopenthixol", "Amisulpride", "Remifentanil", "Regadenoson", 
           "Cyclophosphamide", "Ifosfamide", "Irbesartan", "Baclofen", "Dyphylline", "Aminophylline", "Isoprenaline", 
           "Nesiritide", "Lanreotide", "Aprotinin", "Tadalafil", "Mexiletine", "Lorazepam", "Isoflurane", "Aripiprazole", 
           "Isradipine", "Bosentan", "Atropine", "Mirtazapine", "Procaine", "Epirubicin", "Citalopram", "Eletriptan", 
           "Amsacrine", "Olanzapine", "Diltiazem", "Fluoxetine", "Duloxetine", "Haloperidol", "Risperidone", "Trifluoperazine", 
           "Vardenafil", "Ketoconazole", "Escitalopram", "Quetiapine", "Bepridil", "Ergometrine", "Paliperidone", "Pregabalin", 
           "Foscarnet", "Norfloxacin", "Dasatinib", "Nifedipine", "Ofloxacin", "Arsenic trioxide", "Tranexamic acid", "Fosinopril", 
           "Valganciclovir", "Labetalol", "Theophylline", "Oxtriphylline", "Dofetilide", "Epinephrine", "Imipramine", "Sildenafil", 
           "Memantine", "Erythromycin", "Dipyridamole", "Telithromycin", "Clarithromycin", "Romidepsin", "Dopamine", "Digoxin", 
           "Tacrolimus", "Adenosine", "Ciprofloxacin", "Ganciclovir", "Levofloxacin", "Doxorubicin", "Nitrous oxide", "Protamine", 
           "Iothalamic acid", "Ioxaglic acid", "Cathine", "Gadoversetamide", "Calcitonin porcine", "Parnaparin", "Reviparin", 
           "Gadodiamide", "Diatrizoate", "Ertapenem", "Ioversol", "Iopromide", "Tramazoline", "Iohexol", "Iopamidol", "Clopenthixol", 
           "Tigecycline", "Gadobenic acid", "Gadofosveset trisodium", "Coenzyme M", "Vidarabine", "Phenylbutyric acid", "Esatenolol", 
           "Amphotericin B", "Busulfan", "Micafungin", "Methoxsalen", "Thiotepa", "Voriconazole", "Praziquantel", "Halofantrine", 
           "Cevimeline", "Trospium", "Nabilone", "Vinorelbine", "Esmolol", "Ropivacaine", "Bupivacaine", "Levobupivacaine", "Prilocaine", 
           "Leuprolide", "Goserelin", "Degarelix", "Gliclazide", "Tiagabine", "Nabumetone", "Ketorolac", "Fenoprofen", "Mefenamic acid", 
           "Oxaprozin", "Etoricoxib", "Dexketoprofen", "Dexmedetomidine", "Palonosetron", "Dolasetron", "Granisetron", "Alosetron", 
           "Capecitabine", "Indapamide", "Neostigmine", "Salmon calcitonin", "Milrinone", "Anagrelide", "Nicorandil", "Chlorthalidone", 
           "Cinacalcet", "Nitroprusside", "Nitroglycerin", "Succinylcholine", "Hyoscyamine", "Tropicamide", "Oxybutynin", "Pilocarpine", 
           "Brimonidine", "Apraclonidine", "Periciazine", "Minoxidil", "Methylphenidate", "Modafinil", "Diethylpropion", "Carmustine", 
           "Interferon alfa-2b", "Isotretinoin", "Eplerenone", "Betaxolol", "Sotalol", "Carteolol", "Orciprenaline", "Levobunolol", 
           "Raltitrexed", "Bexarotene", "Phenylephrine", "Phentolamine", "Tamsulosin", "Carbamoylcholine", "Moxifloxacin", "Sibutramine", 
           "Dexmethylphenidate", "Thioridazine", "Zuclopenthixol", "Mycophenolic acid", "Hydromorphone", "Butorphanol", "Sufentanil", 
           "Remifentanil", "Bemiparin", "Risedronic acid", "Paricalcitol", "Doxercalciferol", "Levonorgestrel", "Triamcinolone", 
           "Ondansetron", "Naratriptan", "Rizatriptan", "Frovatriptan", "Oxytocin", "Amifostine", "Cyclophosphamide", "Ifosfamide", 
           "Pindolol", "Salbutamol", "Celiprolol", "Irbesartan", "Pentoxifylline", "Tinzaparin", "Dalteparin", "Tolcapone", 
           "Hydroxocobalamin", "Dipivefrin", "Rivastigmine", "Mivacurium", "Febuxostat", "Pentostatin", "Ziconotide", "Baclofen", 
           "Aminophylline", "Isoprenaline", "Glatiramer", "Nesiritide", "Dextropropoxyphene", "Zanamivir", "Deferoxamine", "Muromonab", 
           "Imiquimod", "Misoprostol", "Lanreotide", "Aprotinin", "Cyclobenzaprine", "Gadopentetic acid", "Dacarbazine", 
           "Isosorbide mononitrate", "Doxapram", "Zolpidem", "Ginkgo biloba", "Valdecoxib", "Dorzolamide", "Brinzolamide", "Mexiletine", 
           "Nadroparin", "Lorazepam", "Etomidate", "Meprobamate", "Isoflurane", "Flumazenil", "Estazolam", "Aripiprazole", 
           "Phenylpropanolamine", "Prochlorperazine", "Tizanidine", "Maprotiline", "Pergolide", "Dosulepin", "Thalidomide", "Ropinirole", 
           "Enflurane", "Procarbazine", "Tramadol", "Esketamine", "Ketamine", "Chenodeoxycholic acid", "Human calcitonin", 
           "Cyproterone acetate", "Calcitriol", "Sevoflurane", "Desflurane", "Ephedrine", "Dextroamphetamine", "Glimepiride", "Atenolol", 
           "Bosentan", "Atropine", "Fenofibrate", "Glipizide", "Mirtazapine", "Pseudoephedrine", "Tiotropium", "Tretinoin", "Zoledronic acid", 
           "Epirubicin", "Idarubicin", "Levocetirizine", "Repaglinide", "Fluvoxamine", "Citalopram", "Eletriptan", "Reboxetine", "Ziprasidone", 
           "Cabergoline", "Amsacrine", "Zolmitriptan", "Amitriptyline", "Methadone", "Olanzapine", "Cetirizine", "Diltiazem", "Protriptyline", 
           "Mefloquine", "Clozapine", "Timolol", "Amlodipine", "Betamethasone", "Loratadine", "Fluoxetine", "Duloxetine", "Lenalidomide", 
           "Buspirone", "Alitretinoin", "Toremifene", "Nortriptyline", "Chloroquine", "Bisoprolol", "Nicardipine", "Trazodone", "Galantamine", 
           "Paroxetine", "Trimipramine", "Risperidone", "Scopolamine", "Oxcarbazepine", "Alfentanil", "Fentanyl", "Propofol", "Diazepam", 
           "Clomifene", "Methylprednisolone", "Desloratadine", "Felodipine", "Ketoconazole", "Selegiline", "Pimozide", "Sertraline", 
           "Bicalutamide", "Doxepin", "Terazosin", "Escitalopram", "Propafenone", "Domperidone", "Bromocriptine", "Quetiapine", "Clomipramine", 
           "Bepridil", "Posaconazole", "Darunavir", "Paliperidone", "Itraconazole", "Citric acid", "Pregabalin", "Sulindac", "Etodolac", 
           "Venlafaxine", "Gefitinib", "Erlotinib", "Riluzole", "Donepezil", "Leflunomide", "Rabeprazole", "Mitoxantrone", "Dasatinib", 
           "Octreotide", "Azithromycin", "Tamoxifen", "Nitrofurantoin", "Eprosartan", "Telmisartan", "Lomefloxacin", "Nifedipine", "Ofloxacin", 
           "Arsenic trioxide", "Sparfloxacin", "Fusidic acid", "Ramipril", "Tranexamic acid", "Fosinopril", "Trandolapril", "Benazepril", 
           "Lisinopril", "Perindopril", "Cilazapril", "Valganciclovir", "Celecoxib", "Flurbiprofen", "Meloxicam", "Promethazine", "Sunitinib", 
           "Nilotinib", "Teniposide", "Prednisone", "Sumatriptan", "Prednisolone", "Ursodeoxycholic acid", "Oseltamivir", "Pantoprazole", 
           "Piroxicam", "Pemetrexed", "Hydrocortisone", "Naproxen", "Quinapril", "Hydrochlorothiazide", "Ibuprofen", "Dexamethasone", 
           "Dexibuprofen", "Theophylline", "Valproic acid", "Cefotaxime", "Fluorouracil", "Enalapril", "Ketoprofen", "Dofetilide", "Metoprolol", 
           "Amiodarone", "Bupropion", "Varenicline", "Reserpine", "Disopyramide", "Quinine", "Doxazosin", "Efavirenz", "Epinephrine", "Rocuronium", 
           "Ranitidine", "Quinidine", "Amantadine", "Amphetamine", "Nicotine", "Lansoprazole", "Imipramine", "Clonidine", "Irinotecan", "Salmeterol", 
           "Formoterol", "Melphalan", "Desipramine", "Sildenafil", "Daunorubicin", "Furosemide", "Famotidine", "Memantine", "Indomethacin", 
           "Cyclosporine", "Pravastatin", "Bortezomib", "Erythromycin", "Caspofungin", "Dipyridamole", "Telithromycin", "Atorvastatin", "Fluvastatin", 
           "Rosuvastatin", "Pioglitazone", "Paclitaxel", "Docetaxel", "Liothyronine", "Levothyroxine", "Dextrothyroxine", "Diclofenac", "Alprostadil",
           "Benzylpenicillin", "Norepinephrine", "Cisplatin", "Digoxin", "Dinoprostone", "Levocarnitine", "Tacrolimus", "Imatinib", "Cladribine", 
           "Adenosine", "Fludarabine", "Gemcitabine", "Zalcitabine", "Niacin", "Ranolazine", "Ciprofloxacin", "Verapamil", "Ganciclovir", "Procainamide",
           "Levofloxacin", "Methylene blue", "Sorafenib", "Carbamazepine", "Doxorubicin", "Losartan", "Topiramate", "Azathioprine", "Protamine", 
           "Daptomycin", "Rasburicase", "Busulfan", "Voriconazole", "Dolasetron", "Capecitabine", "Milrinone", "Anagrelide", "Hyoscyamine", 
           "Periciazine", "Interferon alfa-2b", "Moxifloxacin", "Felbamate", "Cyclophosphamide", "Ifosfamide", "Irbesartan", "Rasagiline", 
           "Lanreotide", "Aprotinin", "Isosorbide mononitrate", "Mexiletine", "Amoxapine", "Sevoflurane", "Atropine", "Mirtazapine", "Duloxetine", 
           "Lenalidomide", "Galantamine", "Paroxetine", "Propofol", "Conivaptan", "Sertraline", "Propafenone", "Ixabepilone", "Pregabalin", 
           "Arsenic trioxide", "Fosinopril", "Trandolapril", "Pemetrexed", "Pramipexole", "Memantine", "Telithromycin", "Romidepsin", "Tacrolimus", 
           "Gemcitabine"]


In [ ]:
list_of_pos_neg = HF_cont+exc_sid+HF_ind

In [153]:
list_uknown_med = [item for item in list_all_med if item not in list_of_pos_neg]

In [154]:
med_embeddigns_pos = [np.concatenate((medication_embeddings[key],diseases_embeddings.get('atrial fibrillation'))) for key in HF_ind if key in medication_embeddings]
med_embeddigns_neg = [np.concatenate((medication_embeddings[key],diseases_embeddings.get('atrial fibrillation'))) for key in HF_cont if key in medication_embeddings]
uknown_medication_embeddings = [np.concatenate((medication_embeddings[key],diseases_embeddings.get('atrial fibrillation'))) for key in list_uknown_med if key in medication_embeddings]
HF_embeddings = diseases_embeddings.get('atrial fibrillation')

In [155]:
med_disease_pos = pd.DataFrame(np.array(med_embeddigns_pos))
med_disease_pos['Label'] = 1
med_disease_neg = pd.DataFrame(np.array(med_embeddigns_neg))
med_disease_neg['Label'] = 0

In [156]:
med_disease_pos_neg = pd.concat([med_disease_pos,med_disease_neg])

In [157]:
med_disease_pos_neg.reset_index(drop=True, inplace=True)

In [165]:
uknown_med_names= [key for key in list_uknown_med if key in medication_embeddings]
uknown_med_embeddigns_df = pd.DataFrame(np.array(uknown_medication_embeddings))

In [161]:
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score
import random 

random.seed(123456)
np.random.seed(123456)
X = med_disease_pos_neg.iloc[:,0:-1]
y = med_disease_pos_neg.iloc[:,-1]
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=20)

In [190]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score
# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 5, 10, 50, 100, 200, 300, 500],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'linear']
}

# Initialize the classifier
random.seed(123)
svm = SVC(probability=True, random_state=42)

# Initialize GridSearchCV
random.seed(123)
grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy', verbose=2)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters and best estimator
best_params = grid_search.best_params_
best_svm = grid_search.best_estimator_

print("Best parameters found: ", best_params)

# Evaluate the best estimator
y_pred_svm = best_svm.predict(X_test)
y_pred_proba_svm = best_svm.predict_proba(X_test)[:, 1]
print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print("ROC AUC:", roc_auc_score(y_test, 1-y_pred_proba_svm))


Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best parameters found:  {'C': 500, 'gamma': 1, 'kernel': 'rbf'}
Accuracy: 0.7222222222222222
ROC AUC: 0.8375


In [209]:
svm_my_dict = {}
for i in range(0,uknown_med_embeddigns_df.shape[0]):
    y_pred_svm_p = svm.predict(uknown_med_embeddigns_df.iloc[i:i+1,:])
    y_pred_proba_svm_p = svm.predict_proba(uknown_med_embeddigns_df.iloc[i:i+1,:])[:, 1]
    print(uknown_med_names[i],y_pred_svm_p[0],1-y_pred_proba_svm_p[0])
    if y_pred_svm_p[0]>0:
        svm_my_dict[uknown_med_names[i]] =1-y_pred_proba_svm_p[0]

pos_prd_prob_svm = dict(sorted(svm_my_dict.items(), key=lambda item: item[1],reverse=True))

Diphenoxylate 0 0.4670037219025083
Plazomicin 1 0.5398510933186891
Tetracaine 0 0.46193681989192337
Roflumilast 0 0.3902303691631003
Gonadorelin 0 0.34760929448349165
Empagliflozin 0 0.4211829809192943
Anisotropine methylbromide 1 0.5428347228630332
Clemastine 1 0.6026741043911448
Amiloride 1 0.5
Fenfluramine 0 0.4314006872303029
Phentermine 0 0.4846903145622189
Mazindol 0 0.3572732975397447
Methsuximide 1 0.5433886662227154
Yohimbine 0 0.40452571451500063
Insulin lispro 1 0.5148704417182488
Sodium citrate 0 0.3713329328689806
Histamine 0 0.4338310286818269
Vorinostat 0 0.4891846038046914
Antihemophilic factor, human recombinant 1 0.7066992180922664
Viloxazine 0 0.4916886021289516
Mirabegron 1 0.5295539162244003
Mycophenolate mofetil 0 0.38778325726311624
Captopril 1 0.6569027850123085
Trilostane 1 0.5143513685032102
Mafenide 0 0.4686895693651829
Nystatin 0 0.3685666193359337
Ferrous sulfate anhydrous 1 0.5252404762866225
Tranilast 1 0.5853533741242969
Ropeginterferon alfa-2b 1 0.60508